In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from selenium import webdriver

In [2]:
res=requests.get("https://tw.allianzgi.com/zh-tw/insights/market-insight")

soup=BeautifulSoup(res.text, "html.parser")

In [3]:
res

<Response [200]>

In [4]:
soup


<!DOCTYPE html>

<!--otcs is Not Null-->
<!--One Trust Cookie can be used for this site-->
<!--OneTrustScriptURL is Not Empty-->
<!--If User has accepted Sitecore global analytics cookie in his preference list-->
<!--
        <div>  SAMLart =  </div><div>  extPartner =  </div><div>  SAMLResponse =  </div><div>  webAppTargetUrl =  </div><div>  selfRedirected =  </div>
        -->
<html class="no-js" lang="zh-TW">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!--Page Metadata-->
<title>財富趨勢掃描-每週提供國際熱錢動態，掌握投資趨勢-安聯投信</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="none" name="msapplication-config"/>
<meta content="zh-TW" http-equiv="Content-Language"/>
<meta content="zh-TW" name="language"/>
<meta content="tw" name="Country"/>
<meta content="安聯投信財富趨勢掃描，每週發表國際熱錢走向與後市展望，此外不定期出刊的安聯投信投資透視，更是您不容錯過的投資理財入門，讓您掌握最新最快的財經訊息。
" name="description"/>
<meta content="基金, 定期定額, 投資, 理財" name="keywords"/>
<link 

In [41]:
titles=[]
articles=soup.select(".c-heading.c-agi-tile__headline.c-heading--link-black a")
for each_title in articles:
    titles.append(each_title.text.replace("\r\n","").strip())
titles

['【安聯投信投資市場月報】2021年05月投資市場月報 - 全速前進',
 '未來路線圖：介紹我們的投資主題',
 '【安聯投信投資透視】中國：世界股市中的「沉睡巨人」',
 '【2021春季投資論壇摘要 - 復甦路上的攻與守】美國及亞洲總經評論與投資策略',
 '【2021春季投資論壇摘要 - 復甦路上的攻與守】展望全球主題趨勢股票投資',
 '【安聯投信投資市場月報】2021年04月投資市場月報 - 膨脹的市場，膨脹的擔憂，抑或只是通貨膨脹？',
 '【安聯投信市場評析】等待政策空窗期 短期不確定影響',
 '【安聯投信投資市場月報】2021年03月投資市場月報 - 一時過熱',
 '【安聯投信事件評析】市場震盪 逢低布局']

In [43]:
dates=[]
time=soup.select(".c-copy.c-copy--medium.c-agi-tile__author")
for each_date in time:
    dates.append(each_date.text.replace("\r\n","").strip())
dates

['4 小時前',
 '2021/05/04',
 '2021/04/19',
 '2021/04/06',
 '2021/04/06',
 '2021/04/06',
 '2021/03/10',
 '2021/03/09',
 '2021/02/26']

In [42]:
url="https://tw.allianzgi.com/zh-tw/insights/market-insight"
dates=[]
titles=[]
links=[]
# for times in range(20):
res = requests.get(url)
soup=BeautifulSoup(res.text, "html.parser")
articles=soup.select(".c-heading.c-agi-tile__headline.c-heading--link-black a")
time = soup.select(".c-copy.c-copy--medium.c-agi-tile__author")
#     paging1=soup.select(".btn-group.btn-group-paging a")
#     next_url = "https://tw.allianzgi.com/" + paging1[1]["href"]
#     url=next_url
for each_date in time:
    dates.append(each_date.text.replace("\r\n","").strip())
for each_title in articles:
    titles.append(each_title.text.replace("\r\n","").strip())
for each_title in articles:
    links.append("https://tw.allianzgi.com/"+each_title["href"] )
        
df = pd.DataFrame(
{
    '日期':dates,
    '標題':titles,
    '內容連結':links
})

# df1.to_csv('ptt fund.csv',encoding='utf_8')
df

,日期,標題,內容連結
0,4 小時前,【安聯投信投資市場月報】2021年05月投資市場月報 - 全速前進,https://tw.allianzgi.com//zh-tw/home/insights/...
1,2021/05/04,未來路線圖：介紹我們的投資主題,https://tw.allianzgi.com//zh-tw/home/insights/...
2,2021/04/19,【安聯投信投資透視】中國：世界股市中的「沉睡巨人」,https://tw.allianzgi.com//zh-tw/home/insights/...
3,2021/04/06,【2021春季投資論壇摘要 - 復甦路上的攻與守】美國及亞洲總經評論與投資策略,https://tw.allianzgi.com//zh-tw/home/insights/...
4,2021/04/06,【2021春季投資論壇摘要 - 復甦路上的攻與守】展望全球主題趨勢股票投資,https://tw.allianzgi.com//zh-tw/home/insights/...
5,2021/04/06,【安聯投信投資市場月報】2021年04月投資市場月報 - 膨脹的市場，膨脹的擔憂，抑或只是通...,https://tw.allianzgi.com//zh-tw/home/insights/...
6,2021/03/10,【安聯投信市場評析】等待政策空窗期 短期不確定影響,https://tw.allianzgi.com//zh-tw/home/insights/...
7,2021/03/09,【安聯投信投資市場月報】2021年03月投資市場月報 - 一時過熱,https://tw.allianzgi.com//zh-tw/home/insights/...
8,2021/02/26,【安聯投信事件評析】市場震盪 逢低布局,https://tw.allianzgi.com//zh-tw/home/insights/...
